In [18]:
import torch.nn as nn
import random

def generate_random_network():
    layers = []
    input_shape = [random.randint(1, 10), random.randint(10, 200), random.randint(1, 20)]
    output_shape = input_shape.copy()
    num_layers = random.randint(1, 10)
    
    for _ in range(num_layers):
        layer_type = random.choice(['Linear', 'Conv2d', 'LSTM'])
        if layer_type == 'Linear':
            in_features = output_shape[-1]
            out_features = random.randint(1, 100)
            layers.append(f'nn.Linear({in_features}, {out_features})')
            output_shape[-1] = out_features
        elif layer_type == 'Conv2d':
            in_channels = output_shape[0]
            out_channels = random.randint(1, 64)
            kernel_size = random.randint(1, 5)
            layers.append(f'nn.Conv2d({in_channels}, {out_channels}, {kernel_size})')
            output_shape[0] = out_channels
            output_shape[1] = max(1, output_shape[1] - kernel_size + 1)
            output_shape[2] = max(1, output_shape[2] - kernel_size + 1)
        elif layer_type == 'LSTM':
            input_dim = output_shape[-1]
            hidden_dim = random.randint(1, 100)
            layers.append(f'nn.LSTM({input_dim}, {hidden_dim})')
            output_shape[-1] = hidden_dim

    return layers, input_shape, output_shape

def generate_data(num_samples=1000):
    data = []
    for _ in range(num_samples):
        layers, input_shape, output_shape = generate_random_network()
        description = f'Input shape: {input_shape}, Output shape: {output_shape}'
        data.append((layers, description))
    return data

data = generate_data()


In [19]:
import pandas as pd

df= pd.DataFrame(data)

df.head()

,0,1
0,"[nn.Linear(16, 64), nn.LSTM(64, 72), nn.Linear...","Input shape: [7, 21, 16], Output shape: [7, 21..."
1,"[nn.Linear(5, 82), nn.Linear(82, 20)]","Input shape: [4, 155, 5], Output shape: [4, 15..."
2,"[nn.LSTM(6, 41), nn.Linear(41, 78), nn.Conv2d(...","Input shape: [6, 37, 6], Output shape: [24, 34..."
3,"[nn.Linear(14, 17), nn.Conv2d(7, 33, 2)]","Input shape: [7, 156, 14], Output shape: [33, ..."
4,"[nn.Linear(15, 77), nn.LSTM(77, 97)]","Input shape: [7, 135, 15], Output shape: [7, 1..."


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Tokenization
def tokenize_architecture(text):
    return text.split(', ')

def tokenize_description(text):
    return text.split()

# Custom Dataset
class NeuralNetworkDataset(Dataset):
    def __init__(self, data, src_vocab, trg_vocab):
        self.data = data
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src, trg = self.data[idx]
        src_tokens = [self.src_vocab[token] for token in tokenize_architecture(str(src))]
        trg_tokens = [self.trg_vocab[token] for token in tokenize_description(trg)]
        return torch.tensor(src_tokens), torch.tensor(trg_tokens)

# Vocabulary
def build_vocab(data, tokenizer):
    vocab = build_vocab_from_iterator((tokenizer(sentence) for sentence in data), specials=["<unk>", "<pad>", "<sos>", "<eos>"])
    vocab.set_default_index(vocab["<unk>"])
    return vocab

# Collate function
def collate_fn(batch):
    src_batch, trg_batch = [], []
    for src_sample, trg_sample in batch:
        src_batch.append(torch.cat([torch.tensor([src_vocab['<sos>']]), src_sample, torch.tensor([src_vocab['<eos>']])], dim=0))
        trg_batch.append(torch.cat([torch.tensor([trg_vocab['<sos>']]), trg_sample, torch.tensor([trg_vocab['<eos>']])], dim=0))
    
    src_batch = pad_sequence(src_batch, padding_value=src_vocab['<pad>'])
    trg_batch = pad_sequence(trg_batch, padding_value=trg_vocab['<pad>'])
    
    return src_batch, trg_batch

# Generate data
data = generate_data()

# Build vocabularies
src_sentences = [str(layers) for layers, _ in data]
trg_sentences = [description for _, description in data]
src_tokenizer = get_tokenizer(tokenize_architecture)
trg_tokenizer = get_tokenizer(tokenize_description)

src_vocab = build_vocab(src_sentences, src_tokenizer)
trg_vocab = build_vocab(trg_sentences, trg_tokenizer)

# Dataset and DataLoader
dataset = NeuralNetworkDataset(data, src_vocab, trg_vocab)
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=32, collate_fn=collate_fn)

# Model Definition
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, trg.shape[1], trg_vocab_size).to(self.device)
        
        hidden, cell = self.encoder(src)
        
        input = trg[0, :]
        
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        
        return outputs

# Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trg_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=trg_vocab["<pad>"])

def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
    
    return epoch_loss / len(iterator)

# Example training loop
N_EPOCHS = 10
CLIP = 1

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.3f}')


Epoch: 1, Train Loss: 3.953
Epoch: 2, Train Loss: 2.885
Epoch: 3, Train Loss: 2.471
Epoch: 4, Train Loss: 2.334
Epoch: 5, Train Loss: 2.290
Epoch: 6, Train Loss: 2.267
Epoch: 7, Train Loss: 2.257
Epoch: 8, Train Loss: 2.251
Epoch: 9, Train Loss: 2.250
Epoch: 10, Train Loss: 2.243


In [27]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    predictions, targets = [], []
    
    with torch.no_grad():
        for i, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            
            output = model(src, trg, 0)
            
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
            
            predictions.extend(output.argmax(1).cpu().numpy())
            targets.extend(trg.cpu().numpy())
    
    accuracy = accuracy_score(targets, predictions)
    f1 = f1_score(targets, predictions, average='weighted')
    
    return epoch_loss / len(iterator), accuracy, f1



# Example training loop
N_EPOCHS = 10
CLIP = 1

for epoch in range(N_EPOCHS):
    valid_loss, valid_accuracy, valid_f1 = evaluate(model, valid_loader, criterion)
    print(f'Epoch: {epoch+1}, Valid Loss: {valid_loss:.3f}, Valid Accuracy: {valid_accuracy:.3f}, Valid F1 Score: {valid_f1:.3f}')

Epoch: 1, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 2, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 3, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 4, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 5, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 6, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 7, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 8, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 9, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
Epoch: 10, Valid Loss: 2.298, Valid Accuracy: 0.478, Valid F1 Score: 0.461
